In [1]:
# Data handling
import pandas as pd
import numpy as np
from tqdm import tqdm

# RDKit for chemistry
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

print("All libraries loaded successfully ✅")


ModuleNotFoundError: No module named 'rdkit'

In [ ]:
import pubchempy as pcp
import pandas as pd
from tqdm import tqdm

# List to store compounds
compounds = []

# PubChem search for "analgesic"
results = pcp.get_compounds('analgesic', 'name', listkey_count=200)

for c in tqdm(results[:200], desc="Fetching compounds"):
    if c.isomeric_smiles:
        compounds.append({
            "Name": c.iupac_name if c.iupac_name else c.synonyms[0],
            "CID": c.cid,
            "SMILES": c.isomeric_smiles
        })

# Convert to DataFrame
df = pd.DataFrame(compounds)
df.to_csv("analgesics_200.csv", index=False)
print("Saved", len(df), "analgesics to CSV")
df.head()


In [ ]:
import pandas as pd

# Start with small real set
base_data = {
    "Name": ["Aspirin", "Ibuprofen", "Paracetamol", "Morphine", "Fentanyl"],
    "CID": [2244, 3672, 1983, 5288826, 3345],
    "SMILES": [
        "CC(=O)OC1=CC=CC=C1C(=O)O",  # Aspirin
        "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O",  # Ibuprofen
        "CC(=O)NC1=CC=C(C=C1)O",  # Paracetamol
        "CN1CC[C@]23[C@@H]4[C@H]1CC5=C2C(=C(C=C5)O)O[C@H]4C3",  # Morphine
        "CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3"  # Fentanyl
    ]
}

df_base = pd.DataFrame(base_data)

# Duplicate and modify slightly to make 100 molecules
df_list = []
for i in range(20):  # 20*5 = 100
    temp = df_base.copy()
    temp["Name"] = temp["Name"] + f"_v{i+1}"
    temp["CID"] = temp["CID"] + i*10000  # make CID unique
    temp["SMILES"] = temp["SMILES"]  # keep SMILES same for demo
    df_list.append(temp)

df_100 = pd.concat(df_list, ignore_index=True)

# Save to CSV
df_100.to_csv("analgesics_100.csv", index=False)
print("Created 100-molecule analgesics dataset ✅")
df_100.head()


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from tqdm import tqdm
import pandas as pd

# Load the CSV
df = pd.read_csv("analgesics_100.csv")

# Create descriptor calculator
descriptor_names = [desc[0] for desc in Descriptors._descList]
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

# Compute descriptors
features = []
for smi in tqdm(df["SMILES"], desc="Computing descriptors"):
    mol = Chem.MolFromSmiles(smi)
    if mol:
        features.append(calculator.CalcDescriptors(mol))
    else:
        # Handle invalid SMILES
        features.append([None]*len(descriptor_names))

# Convert to DataFrame
X = pd.DataFrame(features, columns=descriptor_names)
X.to_csv("analgesics_100_descriptors.csv", index=False)
print("Descriptors computed and saved. Shape:", X.shape)


In [ ]:
import numpy as np

# For demo, let’s assign classes manually
# 0 = NSAID (like Aspirin, Ibuprofen, Paracetamol)
# 1 = Opioid (like Morphine, Fentanyl, Tramadol if present)

# Create empty label column
df["Class"] = np.nan

# Assign classes based on original name
for i, row in df.iterrows():
    if "Aspirin" in row["Name"] or "Ibuprofen" in row["Name"] or "Paracetamol" in row["Name"]:
        df.at[i, "Class"] = 0
    else:
        df.at[i, "Class"] = 1

# Make sure it's integer type
df["Class"] = df["Class"].astype(int)

# Save labeled dataset
df.to_csv("analgesics_100_labeled.csv", index=False)
df.head(10)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load descriptors and labels
X = pd.read_csv("analgesics_100_descriptors.csv")
y = pd.read_csv("analgesics_100_labeled.csv")["Class"]

# Split data into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on test set
y_pred = clf.predict(X_test)

# Evaluate
print("Classification Report:\n")
print(classification_report(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get feature importances
importances = clf.feature_importances_
features = X.columns

# Create DataFrame for plotting
feat_df = pd.DataFrame({"Feature": features, "Importance": importances})
feat_df = feat_df.sort_values(by="Importance", ascending=False).head(10)  # top 10

# Plot
plt.figure(figsize=(10,6))
sns.barplot(x="Importance", y="Feature", data=feat_df)
plt.title("Top 10 Descriptor Importances")
plt.show()


In [ ]:
%pip install seaborn


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get feature importances
importances = clf.feature_importances_
features = X.columns

# Create DataFrame for plotting
feat_df = pd.DataFrame({"Feature": features, "Importance": importances})
feat_df = feat_df.sort_values(by="Importance", ascending=False).head(10)  # top 10

# Plot
plt.figure(figsize=(10,6))
sns.barplot(x="Importance", y="Feature", data=feat_df)
plt.title("Top 10 Descriptor Importances")
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["NSAID", "Opioid"],
            yticklabels=["NSAID", "Opioid"])
plt.xlabel("Predicted Class")
plt.ylabel("Actual Class")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
import sys
print("Current Python:", sys.executable)

In [ ]:
import qiskit
print(qiskit.__version__)  # Should show 2.2.0

In [ ]:
!pip install pandas numpy qiskit-aer qiskit-machine-learning matplotlib seaborn

In [ ]:
# First, let's check what versions we have and what's available
import qiskit
import qiskit_aer
import qiskit_machine_learning

print("Qiskit version:", qiskit.__version__)
print("Qiskit Aer version:", qiskit_aer.__version__)
print("Qiskit Machine Learning version:", qiskit_machine_learning.__version__)

# Check what's available in the kernels module
print("\nAvailable in qiskit_machine_learning.kernels:")
import qiskit_machine_learning.kernels as kernels
print(dir(kernels))

# Try different import paths
try:
    from qiskit_machine_learning.kernels import QuantumKernel
    print("✅ QuantumKernel imported successfully")
except ImportError as e:
    print("❌ QuantumKernel import failed:", e)
    
    try:
        from qiskit_machine_learning.kernels import FidelityQuantumKernel as QuantumKernel
        print("✅ FidelityQuantumKernel imported as QuantumKernel")
    except ImportError as e2:
        print("❌ FidelityQuantumKernel import failed:", e2)
        
        try:
            from qiskit_machine_learning.kernels.quantum_kernel import QuantumKernel
            print("✅ QuantumKernel imported from quantum_kernel module")
        except ImportError as e3:
            print("❌ All QuantumKernel imports failed:", e3)

In [ ]:
# Install packages (run this first if needed):
# !pip install qiskit-aer qiskit-machine-learning

import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap

# Create sample data using NumPy (replace this with your actual dataset)
np.random.seed(42)  # For reproducible results
X = np.random.rand(10, 2)  # 10 samples, 2 features

print("Sample data:")
print("First 5 rows:")
print(X[:5])
print(f"Data shape: {X.shape}")

# Small quantum kernel demo - Updated for current Qiskit versions
backend = Aer.get_backend('aer_simulator')
feature_map = ZZFeatureMap(feature_dimension=min(2, X.shape[1]), reps=1)

# Updated: Using FidelityQuantumKernel (new name for QuantumKernel)
qkernel = FidelityQuantumKernel(feature_map=feature_map)

# Take first 5 samples for demo
X_small = X[:5]  # Using NumPy slicing
print(f"\nUsing {X_small.shape[0]} samples for quantum kernel computation...")

# Compute quantum kernel matrix
kernel_matrix = qkernel.evaluate(X_small)
print("Quantum kernel computed ✅")
print("\nKernel Matrix:")
print(kernel_matrix)

# Optional: Display kernel matrix properties
print(f"\nKernel matrix shape: {kernel_matrix.shape}")
print(f"Kernel matrix diagonal (should be close to 1): {np.diag(kernel_matrix)}")
print(f"Is kernel matrix symmetric? {np.allclose(kernel_matrix, kernel_matrix.T)}")

# Optional: Show feature map details
print(f"\nFeature map details:")
print(f"Number of qubits: {feature_map.num_qubits}")
print(f"Number of parameters: {feature_map.num_parameters}")

In [ ]:
# Now you can use this quantum kernel for machine learning tasks!

# 1. QUANTUM SVM (Support Vector Machine)
from sklearn.svm import SVC
import numpy as np

# Create larger dataset for ML
np.random.seed(42)
X_large = np.random.rand(50, 2)
y_large = (X_large[:, 0] + X_large[:, 1] > 1).astype(int)  # Simple binary classification

# Compute kernel matrix for larger dataset
print("Computing quantum kernel for larger dataset...")
kernel_matrix_large = qkernel.evaluate(X_large)

# Use quantum kernel with SVM
svm_quantum = SVC(kernel='precomputed')
svm_quantum.fit(kernel_matrix_large, y_large)
print("✅ Quantum SVM trained successfully!")

# 2. VISUALIZE KERNEL MATRIX
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Original small kernel matrix
    sns.heatmap(kernel_matrix, annot=True, cmap='viridis', ax=ax1, fmt='.3f')
    ax1.set_title('Quantum Kernel Matrix (5x5)')
    ax1.set_xlabel('Sample Index')
    ax1.set_ylabel('Sample Index')
    
    # Larger kernel matrix (without annotations for clarity)
    sns.heatmap(kernel_matrix_large, cmap='viridis', ax=ax2)
    ax2.set_title('Quantum Kernel Matrix (50x50)')
    ax2.set_xlabel('Sample Index')
    ax2.set_ylabel('Sample Index')
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Install matplotlib and seaborn to see visualizations:")
    print("!pip install matplotlib seaborn")

# 3. COMPARE WITH CLASSICAL KERNELS
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel

# Classical RBF kernel
classical_rbf = rbf_kernel(X[:5])
print(f"\nClassical RBF Kernel (first element): {classical_rbf[0, 1]:.6f}")
print(f"Quantum Kernel (first element): {kernel_matrix[0, 1]:.6f}")

# 4. KERNEL PROPERTIES ANALYSIS
print(f"\n=== KERNEL ANALYSIS ===")
print(f"Quantum Kernel - Min similarity: {np.min(kernel_matrix[kernel_matrix < 1]):.6f}")
print(f"Quantum Kernel - Max similarity: {np.max(kernel_matrix[kernel_matrix < 1]):.6f}")
print(f"Quantum Kernel - Average off-diagonal: {np.mean(kernel_matrix[kernel_matrix < 1]):.6f}")

# Check if kernel is positive semi-definite (required for valid kernels)
eigenvals = np.linalg.eigvals(kernel_matrix)
is_psd = np.all(eigenvals >= -1e-8)  # Allow small numerical errors
print(f"Is positive semi-definite: {is_psd}")
print(f"Smallest eigenvalue: {np.min(eigenvals):.8f}")

print(f"\n🎉 Your quantum kernel is ready for machine learning!")
print(f"💡 Try it with different datasets and compare with classical kernels!")

In [ ]:
pip install matplotlib seaborn

In [ ]:
import matplotlib.pyplot as plt
   import seaborn as sns

In [2]:
# Step 1: Install required packages
!pip install matplotlib seaborn

# Step 2: Import and create visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Recreate the kernel matrix (from your earlier results)
np.random.seed(42)
X_small = np.random.rand(5, 2)
kernel_matrix = qkernel.evaluate(X_small)

# Create visualizations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Visualization 1: Small kernel matrix with values
sns.heatmap(kernel_matrix, annot=True, cmap='viridis', ax=ax1, fmt='.3f', 
            cbar_kws={'label': 'Similarity'})
ax1.set_title('Quantum Kernel Matrix (5×5)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Sample Index')
ax1.set_ylabel('Sample Index')

# Visualization 2: Larger kernel matrix for pattern visualization
X_large = np.random.rand(20, 2)
kernel_matrix_large = qkernel.evaluate(X_large)

sns.heatmap(kernel_matrix_large, cmap='viridis', ax=ax2,
            cbar_kws={'label': 'Similarity'})
ax2.set_title('Quantum Kernel Matrix (20×20)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Sample Index')
ax2.set_ylabel('Sample Index')

plt.tight_layout()
plt.show()

# Additional visualization: Kernel similarity distribution
plt.figure(figsize=(10, 6))
off_diagonal = kernel_matrix[kernel_matrix < 1]
plt.hist(off_diagonal, bins=15, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Distribution of Quantum Kernel Similarities', fontsize=14, fontweight='bold')
plt.xlabel('Similarity Value')
plt.ylabel('Frequency')
plt.axvline(np.mean(off_diagonal), color='red', linestyle='--', 
            label=f'Mean: {np.mean(off_diagonal):.3f}')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print("🎨 Visualizations created successfully!")
print(f"📊 Kernel statistics:")
print(f"   • Mean similarity: {np.mean(off_diagonal):.4f}")
print(f"   • Std deviation: {np.std(off_diagonal):.4f}")
print(f"   • Min similarity: {np.min(off_diagonal):.4f}")
print(f"   • Max similarity: {np.max(off_diagonal):.4f}")

NameError: name 'qkernel' is not defined

In [3]:
# COMPLETE QUANTUM KERNEL ANALYSIS - SELF CONTAINED
# This includes everything from scratch!

import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import rbf_kernel

print("🚀 COMPLETE QUANTUM KERNEL ANALYSIS")
print("=" * 50)

# === STEP 1: CREATE QUANTUM KERNEL ===
print("🔧 Creating quantum kernel...")

# Set up quantum backend and feature map
backend = Aer.get_backend('aer_simulator')
feature_map = ZZFeatureMap(feature_dimension=2, reps=1)

# Create quantum kernel
qkernel = FidelityQuantumKernel(feature_map=feature_map)

print("✅ Quantum kernel created successfully!")

# === STEP 2: BASIC KERNEL TEST ===
print(f"\n📊 BASIC KERNEL TEST")
print("-" * 30)

# Create small test dataset
np.random.seed(42)
X_small = np.random.rand(5, 2)

print("Test data (first 5 samples):")
for i, sample in enumerate(X_small):
    print(f"  Sample {i}: [{sample[0]:.3f}, {sample[1]:.3f}]")

# Compute quantum kernel matrix
kernel_matrix = qkernel.evaluate(X_small)

print(f"\nQuantum Kernel Matrix:")
print(kernel_matrix)

# Analyze kernel properties
print(f"\n📈 Kernel Properties:")
print(f"   • Shape: {kernel_matrix.shape}")
print(f"   • Diagonal values: {np.diag(kernel_matrix)}")
print(f"   • Is symmetric: {np.allclose(kernel_matrix, kernel_matrix.T)}")

off_diagonal = kernel_matrix[kernel_matrix < 1]
print(f"   • Min similarity: {np.min(off_diagonal):.6f}")
print(f"   • Max similarity: {np.max(off_diagonal):.6f}")
print(f"   • Average similarity: {np.mean(off_diagonal):.6f}")

# Check positive semi-definite property
eigenvals = np.linalg.eigvals(kernel_matrix)
is_psd = np.all(eigenvals >= -1e-8)
print(f"   • Positive semi-definite: {is_psd}")
print(f"   • Smallest eigenvalue: {np.min(eigenvals):.8f}")

# === STEP 3: MACHINE LEARNING COMPARISON ===
print(f"\n🤖 MACHINE LEARNING PERFORMANCE TEST")
print("-" * 40)

# Create larger dataset for meaningful ML comparison
n_samples = 60

# Two-class dataset with some complexity
X_class1 = np.random.multivariate_normal([0.3, 0.7], [[0.08, 0.02], [0.02, 0.08]], n_samples//2)
X_class2 = np.random.multivariate_normal([0.7, 0.3], [[0.08, -0.02], [-0.02, 0.08]], n_samples//2)

X_full = np.vstack([X_class1, X_class2])
y_full = np.hstack([np.zeros(n_samples//2), np.ones(n_samples//2)])

# Normalize to [0,1] range (good for quantum circuits)
X_norm = (X_full - X_full.min()) / (X_full.max() - X_full.min())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_full, test_size=0.3, random_state=42)

print(f"Dataset: {len(X_train)} training samples, {len(X_test)} test samples")
print(f"Class distribution: {np.bincount(y_train.astype(int))} (train), {np.bincount(y_test.astype(int))} (test)")

# === QUANTUM SVM ===
print(f"\n🌊 QUANTUM KERNEL SVM")
print("-" * 25)

# Compute quantum kernels
K_train_quantum = qkernel.evaluate(X_train)
K_test_quantum = qkernel.evaluate(X_test, X_train)

# Train quantum SVM
svm_quantum = SVC(kernel='precomputed')
svm_quantum.fit(K_train_quantum, y_train)
y_pred_quantum = svm_quantum.predict(K_test_quantum)
acc_quantum = accuracy_score(y_test, y_pred_quantum)

print(f"✅ Accuracy: {acc_quantum:.4f} ({acc_quantum*100:.1f}%)")

# === CLASSICAL RBF SVM ===
print(f"\n🏛️  CLASSICAL RBF SVM")
print("-" * 25)

svm_rbf = SVC(kernel='rbf', gamma='scale')
svm_rbf.fit(X_train, y_train)
y_pred_rbf = svm_rbf.predict(X_test)
acc_rbf = accuracy_score(y_test, y_pred_rbf)

print(f"✅ Accuracy: {acc_rbf:.4f} ({acc_rbf*100:.1f}%)")

# === LINEAR SVM ===
print(f"\n📏 LINEAR SVM")
print("-" * 25)

svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)
y_pred_linear = svm_linear.predict(X_test)
acc_linear = accuracy_score(y_test, y_pred_linear)

print(f"✅ Accuracy: {acc_linear:.4f} ({acc_linear*100:.1f}%)")

# === FINAL RESULTS ===
print(f"\n🏆 FINAL PERFORMANCE RANKING")
print("=" * 50)

results = [
    ("Quantum Kernel SVM", acc_quantum),
    ("Classical RBF SVM", acc_rbf), 
    ("Linear SVM", acc_linear)
]

# Sort by accuracy (best first)
results.sort(key=lambda x: x[1], reverse=True)

medals = ["🥇", "🥈", "🥉"]
for i, (method, accuracy) in enumerate(results):
    print(f"{medals[i]} {method:<20} {accuracy:.4f} ({accuracy*100:.1f}%)")

# Performance analysis
quantum_vs_best_classical = acc_quantum - max(acc_rbf, acc_linear)
print(f"\n💡 Analysis:")
print(f"   • Quantum advantage: {quantum_vs_best_classical:+.4f}")

if quantum_vs_best_classical > 0.02:
    print("   🌟 SIGNIFICANT quantum advantage!")
elif quantum_vs_best_classical > 0:
    print("   ✨ Quantum shows promise")
else:
    print("   📊 Competitive with classical methods")

# Kernel comparison
sample_data = X_train[:10]
K_quantum_sample = qkernel.evaluate(sample_data)
K_classical_sample = rbf_kernel(sample_data)
correlation = np.corrcoef(K_quantum_sample.flatten(), K_classical_sample.flatten())[0,1]

print(f"   • Kernel correlation with RBF: {correlation:.3f}")
if correlation < 0.7:
    print("   🎯 Quantum kernel explores DIFFERENT similarity patterns!")
else:
    print("   📈 Similar patterns to classical kernels")

print(f"\n🎉 SUCCESS! Your quantum kernel is fully functional and ready for:")
print(f"   • Real-world datasets")
print(f"   • Advanced quantum ML algorithms") 
print(f"   • Research applications")
print(f"   • Scaling to larger problems")

🚀 COMPLETE QUANTUM KERNEL ANALYSIS
🔧 Creating quantum kernel...
✅ Quantum kernel created successfully!

📊 BASIC KERNEL TEST
------------------------------
Test data (first 5 samples):
  Sample 0: [0.375, 0.951]
  Sample 1: [0.732, 0.599]
  Sample 2: [0.156, 0.156]
  Sample 3: [0.058, 0.866]
  Sample 4: [0.601, 0.708]

Quantum Kernel Matrix:
[[1.         0.77000912 0.43130444 0.30048486 0.88193446]
 [0.77000912 1.         0.51138178 0.2420378  0.96832888]
 [0.43130444 0.51138178 1.         0.06227761 0.50476147]
 [0.30048486 0.2420378  0.06227761 1.         0.32637747]
 [0.88193446 0.96832888 0.50476147 0.32637747 1.        ]]

📈 Kernel Properties:
   • Shape: (5, 5)
   • Diagonal values: [1. 1. 1. 1. 1.]
   • Is symmetric: True
   • Min similarity: 0.062278
   • Max similarity: 1.000000
   • Average similarity: 0.565122
   • Positive semi-definite: True
   • Smallest eigenvalue: 0.00837622

🤖 MACHINE LEARNING PERFORMANCE TEST
----------------------------------------
Dataset: 42 trainin